# Tutorial 1: Power Spectrum to Kappa Map Generation

## Learning Objectives
By the end of this tutorial, you will understand:
- The theoretical foundation of gravitational lensing convergence (kappa)
- How cosmic structure power spectra relate to lensing effects
- How to generate realistic kappa maps from power spectra
- Basic visualization and statistical analysis of kappa maps

## Introduction to Gravitational Lensing

Gravitational lensing occurs when light from distant galaxies is deflected by intervening matter (dark matter, galaxies, etc.). The **convergence** (κ, kappa) quantifies how much the cross-sectional area of light bundles changes due to lensing:

- κ > 0: Light bundles converge (magnification)
- κ < 0: Light bundles diverge (demagnification)  
- κ = 0: No lensing effect

The convergence relates directly to the projected matter density along the line of sight.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from scipy import special
from astropy.cosmology import Planck18
from astropy import units as u
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 12

## 1. Understanding Power Spectra

The matter power spectrum P(k) describes how much structure exists at different scales in the universe:
- k: wavenumber (inverse scale)
- P(k): power at scale k

For gravitational lensing, we need the **projected** power spectrum, which integrates the 3D power spectrum along the line of sight.

In [ ]:
def matter_power_spectrum(k, A_s=2.1e-9, n_s=0.965, k_pivot=0.05):
    """
    Simple matter power spectrum model
    
    Parameters:
    k: wavenumber [h/Mpc]
    A_s: scalar amplitude
    n_s: spectral index
    k_pivot: pivot scale [h/Mpc]
    """
    # Scale-invariant part
    P_primordial = A_s * (k / k_pivot)**(n_s - 1)
    
    # Transfer function (simplified)
    q = k / (13.41 * 0.022)  # Simplified shape parameter
    T_k = np.log(1 + 2.34 * q) / (2.34 * q) * \
          (1 + 3.89 * q + (16.1 * q)**2 + (5.46 * q)**3 + (6.71 * q)**4)**(-0.25)
    
    # Growth factor (simplified, z=0)
    D = 1.0
    
    return P_primordial * T_k**2 * D**2

# Generate k range and power spectrum
k_range = np.logspace(-4, 2, 1000)  # h/Mpc
P_k = matter_power_spectrum(k_range)

# Plot the power spectrum
plt.figure(figsize=(10, 6))
plt.loglog(k_range, P_k * k_range**3 / (2 * np.pi**2), 'b-', linewidth=2)
plt.xlabel('Wavenumber k [h/Mpc]')
plt.ylabel('Dimensionless Power Δ²(k)')
plt.title('Matter Power Spectrum')
plt.grid(True, alpha=0.3)
plt.xlim(1e-4, 100)
plt.ylim(1e-6, 1)
plt.show()

print("The matter power spectrum shows:")
print("- Large scales (small k): More power (structures formed early)")
print("- Small scales (large k): Less power (suppressed by various effects)")

## 2. From Power Spectrum to Angular Power Spectrum

For lensing, we need the **angular** power spectrum C_ℓ, which relates to what we observe on the sky. This involves integrating the 3D power spectrum along the line of sight.

In [ ]:
def lensing_efficiency(z, z_source=1.0):
    """
    Lensing efficiency function
    
    Parameters:
    z: redshift of lensing mass
    z_source: redshift of source galaxies
    """
    if z >= z_source:
        return 0.0
    
    chi_lens = Planck18.comoving_distance(z).value  # Mpc
    chi_source = Planck18.comoving_distance(z_source).value
    
    return (chi_source - chi_lens) / chi_source * chi_lens / chi_source

def angular_power_spectrum(ell, z_source=1.0, n_z_bins=50):
    """
    Compute angular power spectrum for lensing convergence
    
    Parameters:
    ell: multipole moment
    z_source: source redshift
    """
    # Critical density in units of h^2 M_sun/Mpc^3
    rho_crit = 2.78e11  # h^2 M_sun/Mpc^3
    Omega_m = 0.31
    
    # Redshift range for integration
    z_max = min(z_source - 0.01, 3.0)
    z_range = np.linspace(0.01, z_max, n_z_bins)
    
    C_l = 0.0
    
    for i, z in enumerate(z_range[:-1]):
        dz = z_range[i+1] - z_range[i]
        
        # Comoving distance
        chi = Planck18.comoving_distance(z).value  # Mpc
        H_z = Planck18.H(z).value  # km/s/Mpc
        
        # Wavenumber corresponding to this ell and chi
        k = (ell + 0.5) / chi  # h/Mpc (approximately)
        
        if k > 0 and k < 100:  # Valid k range
            # Lensing efficiency
            g = lensing_efficiency(z, z_source)
            
            # Growth factor (simplified)
            D_z = 1.0 / (1 + z)  # Linear approximation
            
            # Power spectrum at this z and k
            P_k_z = matter_power_spectrum(k) * D_z**2
            
            # Contribution to C_l
            prefactor = (3/2) * (Planck18.H0.value/100)**2 * Omega_m
            integrand = prefactor**2 * g**2 * P_k_z * (1 + z)**2 / chi**2
            
            C_l += integrand * dz * 299792.458 / H_z  # Convert dz to dchi
    
    return C_l

# Compute angular power spectrum
ell_range = np.logspace(1, 4, 50)  # From ℓ=10 to ℓ=10000
C_ell = [angular_power_spectrum(ell) for ell in ell_range]

# Plot angular power spectrum
plt.figure(figsize=(10, 6))
plt.loglog(ell_range, np.array(C_ell) * ell_range * (ell_range + 1) / (2 * np.pi), 'r-', linewidth=2)
plt.xlabel('Multipole ℓ')
plt.ylabel('ℓ(ℓ+1)C_ℓ/(2π)')
plt.title('Lensing Convergence Angular Power Spectrum')
plt.grid(True, alpha=0.3)
plt.show()

print("The angular power spectrum shows:")
print("- Small ℓ (large scales): Dominated by large-scale structure")
print("- Large ℓ (small scales): More sensitive to small-scale physics")

## 3. Generating Kappa Maps

Now we'll generate actual kappa maps from the angular power spectrum. We'll use HEALPix for efficient spherical map handling.

In [ ]:
def generate_kappa_map(nside=512, lmax=2*512, z_source=1.0, seed=42):
    """
    Generate a random realization of a kappa map
    
    Parameters:
    nside: HEALPix resolution parameter
    lmax: maximum multipole
    z_source: source redshift
    seed: random seed for reproducibility
    """
    np.random.seed(seed)
    
    # Compute angular power spectrum up to lmax
    ell_array = np.arange(2, lmax + 1)  # Start from ℓ=2 (no monopole/dipole)
    C_ell_array = np.array([angular_power_spectrum(ell, z_source) for ell in ell_array])
    
    # Generate random spherical harmonic coefficients
    alm = hp.synalm(C_ell_array, lmax=lmax, new=True)
    
    # Convert to map
    kappa_map = hp.alm2map(alm, nside)
    
    return kappa_map, ell_array, C_ell_array

# Generate kappa map
print("Generating kappa map... (this may take a moment)")
nside = 256  # Lower resolution for faster computation
kappa_map, ell_used, C_ell_used = generate_kappa_map(nside=nside, lmax=2*nside)

print(f"Generated kappa map with:")
print(f"- HEALPix nside: {nside}")
print(f"- Number of pixels: {hp.nside2npix(nside)}")
print(f"- Angular resolution: {hp.nside2resol(nside, arcmin=True):.1f} arcmin")
print(f"- Kappa range: [{np.min(kappa_map):.4f}, {np.max(kappa_map):.4f}]")
print(f"- RMS kappa: {np.std(kappa_map):.4f}")

## 4. Visualizing Kappa Maps

Let's visualize our generated kappa map and understand its statistical properties.

In [ ]:
# Full sky visualization
plt.figure(figsize=(12, 8))
hp.mollview(kappa_map, title='Lensing Convergence (κ) Map', 
           cmap='RdBu_r', min=-0.02, max=0.02, unit='κ')
plt.show()

# Zoom into a specific region
plt.figure(figsize=(10, 8))
hp.gnomview(kappa_map, rot=[0, 0], reso=2, 
           title='Kappa Map - Gnomonic Projection (10° × 10°)',
           cmap='RdBu_r', min=-0.02, max=0.02, unit='κ')
plt.show()

In [ ]:
# Statistical analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Histogram of kappa values
axes[0,0].hist(kappa_map, bins=50, alpha=0.7, density=True, color='skyblue', edgecolor='black')
axes[0,0].axvline(0, color='red', linestyle='--', alpha=0.8, label='κ = 0')
axes[0,0].set_xlabel('Convergence κ')
axes[0,0].set_ylabel('Probability Density')
axes[0,0].set_title('Distribution of Kappa Values')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Power spectrum comparison
kappa_alm = hp.map2alm(kappa_map)
measured_C_ell = hp.alm2cl(kappa_alm)
ell_measured = np.arange(len(measured_C_ell))

axes[0,1].loglog(ell_used, C_ell_used, 'r-', label='Input C_ℓ', linewidth=2)
axes[0,1].loglog(ell_measured[2:len(C_ell_used)+2], measured_C_ell[2:len(C_ell_used)+2], 
                'b--', label='Measured C_ℓ', alpha=0.7)
axes[0,1].set_xlabel('Multipole ℓ')
axes[0,1].set_ylabel('C_ℓ')
axes[0,1].set_title('Power Spectrum: Input vs Measured')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Cumulative distribution
sorted_kappa = np.sort(kappa_map)
cumulative = np.arange(1, len(sorted_kappa) + 1) / len(sorted_kappa)
axes[1,0].plot(sorted_kappa, cumulative, 'g-', linewidth=2)
axes[1,0].axvline(0, color='red', linestyle='--', alpha=0.8)
axes[1,0].set_xlabel('Convergence κ')
axes[1,0].set_ylabel('Cumulative Probability')
axes[1,0].set_title('Cumulative Distribution Function')
axes[1,0].grid(True, alpha=0.3)

# Statistics summary
stats_text = f"""
Kappa Map Statistics:

Mean: {np.mean(kappa_map):.6f}
Std Dev: {np.std(kappa_map):.6f}
Min: {np.min(kappa_map):.6f}
Max: {np.max(kappa_map):.6f}

Percentiles:
5%: {np.percentile(kappa_map, 5):.6f}
25%: {np.percentile(kappa_map, 25):.6f}
50%: {np.percentile(kappa_map, 50):.6f}
75%: {np.percentile(kappa_map, 75):.6f}
95%: {np.percentile(kappa_map, 95):.6f}

Fraction κ > 0: {np.sum(kappa_map > 0) / len(kappa_map):.3f}
Fraction κ < 0: {np.sum(kappa_map < 0) / len(kappa_map):.3f}
"""

axes[1,1].text(0.05, 0.95, stats_text, transform=axes[1,1].transAxes, 
              verticalalignment='top', fontfamily='monospace', fontsize=10)
axes[1,1].set_xlim(0, 1)
axes[1,1].set_ylim(0, 1)
axes[1,1].axis('off')
axes[1,1].set_title('Statistical Summary')

plt.tight_layout()
plt.show()

## 5. Understanding the Physical Interpretation

Let's explore what these kappa values mean physically and how they relate to observable effects.

In [ ]:
def lensing_magnification(kappa, gamma1=0.0, gamma2=0.0):
    """
    Calculate lensing magnification from convergence and shear
    
    For simplicity, we assume shear = 0 (only convergence)
    In reality, shear is also important for lensing
    """
    # Magnification μ = 1 / ((1-κ)² - |γ|²)
    # With γ = 0, this simplifies to μ = 1 / (1-κ)²
    return 1.0 / (1.0 - kappa)**2

def size_scaling_factor(kappa):
    """
    How galaxy sizes are affected by lensing
    Size scales as sqrt(magnification)
    """
    mu = lensing_magnification(kappa)
    return np.sqrt(mu)

# Calculate effects for our kappa map
magnification = lensing_magnification(kappa_map)
size_factor = size_scaling_factor(kappa_map)

# Visualize the effects
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Magnification map
im1 = axes[0,0].hist2d(kappa_map, magnification, bins=50, cmap='viridis')
axes[0,0].plot([-0.05, 0.05], [1, 1], 'r--', linewidth=2, label='No magnification')
axes[0,0].set_xlabel('Convergence κ')
axes[0,0].set_ylabel('Magnification μ')
axes[0,0].set_title('Magnification vs Convergence')
axes[0,0].legend()
plt.colorbar(im1[3], ax=axes[0,0], label='Count')

# Size scaling
im2 = axes[0,1].hist2d(kappa_map, size_factor, bins=50, cmap='plasma')
axes[0,1].plot([-0.05, 0.05], [1, 1], 'r--', linewidth=2, label='No size change')
axes[0,1].set_xlabel('Convergence κ')
axes[0,1].set_ylabel('Size Scaling Factor')
axes[0,1].set_title('Size Change vs Convergence')
axes[0,1].legend()
plt.colorbar(im2[3], ax=axes[0,1], label='Count')

# Distribution of magnifications
axes[1,0].hist(magnification, bins=50, alpha=0.7, density=True, color='orange', edgecolor='black')
axes[1,0].axvline(1, color='red', linestyle='--', alpha=0.8, label='No magnification')
axes[1,0].set_xlabel('Magnification μ')
axes[1,0].set_ylabel('Probability Density')
axes[1,0].set_title('Distribution of Magnifications')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Distribution of size factors
axes[1,1].hist(size_factor, bins=50, alpha=0.7, density=True, color='lightgreen', edgecolor='black')
axes[1,1].axvline(1, color='red', linestyle='--', alpha=0.8, label='No size change')
axes[1,1].set_xlabel('Size Scaling Factor')
axes[1,1].set_ylabel('Probability Density')
axes[1,1].set_title('Distribution of Size Changes')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Physical interpretation:")
print(f"- Average magnification: {np.mean(magnification):.4f}")
print(f"- Average size scaling: {np.mean(size_factor):.4f}")
print(f"- Maximum magnification: {np.max(magnification):.4f}")
print(f"- Fraction of galaxies magnified: {np.sum(magnification > 1) / len(magnification):.3f}")
print(f"- Fraction of galaxies with larger apparent sizes: {np.sum(size_factor > 1) / len(size_factor):.3f}")

## 6. Exercise: Exploring Different Source Redshifts

The amount of lensing depends on the source redshift. Let's explore how kappa maps change for galaxies at different distances.

In [ ]:
# Exercise: Generate kappa maps for different source redshifts
z_sources = [0.5, 1.0, 1.5, 2.0]
colors = ['blue', 'green', 'orange', 'red']

plt.figure(figsize=(12, 8))

for i, z_s in enumerate(z_sources):
    print(f"Computing for z_source = {z_s}...")
    
    # Generate kappa map for this redshift
    kappa_z, _, _ = generate_kappa_map(nside=128, z_source=z_s, seed=42)
    
    # Plot histogram
    plt.hist(kappa_z, bins=30, alpha=0.6, density=True, 
            label=f'z_source = {z_s}', color=colors[i])
    
    print(f"  RMS κ = {np.std(kappa_z):.5f}")

plt.axvline(0, color='black', linestyle='--', alpha=0.8)
plt.xlabel('Convergence κ')
plt.ylabel('Probability Density')
plt.title('Kappa Distributions for Different Source Redshifts')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nKey observations:")
print("- Higher redshift sources experience more lensing")
print("- The width of the kappa distribution increases with source redshift")
print("- This is because light travels through more structure")

## Summary and Key Takeaways

In this tutorial, you learned:

1. **Theoretical Foundation**: Gravitational lensing convergence (κ) quantifies how matter along the line of sight affects light from distant galaxies.

2. **Power Spectrum Connection**: The 3D matter power spectrum P(k) can be projected to create angular power spectra C_ℓ for lensing.

3. **Map Generation**: Random realizations of kappa maps can be generated from power spectra using spherical harmonics.

4. **Physical Effects**: Kappa maps predict observable effects:
   - Magnification: μ = 1/(1-κ)²
   - Size scaling: ∝ √μ
   - Brightness changes

5. **Redshift Dependence**: Sources at higher redshift experience stronger lensing effects.

## Next Steps

In the next tutorial, we'll learn how to:
- Work with realistic galaxy catalogues
- Apply lensing effects to galaxy properties
- Understand observational challenges

## Questions for Further Exploration

1. How would the kappa map change if we used a different cosmology?
2. What happens if we include galaxy-galaxy lensing in addition to cosmic shear?
3. How do observational effects (noise, seeing, etc.) affect our ability to measure these signals?
4. Can you identify regions in the kappa map that would be particularly interesting for follow-up observations?